# Analisis de Sentimiento (simple)

El objetivo de este ejemplo es hacer un *Analisis de Sentimiento* de reseñas de películas, viendo si es positivo o negativo.

Vamos a utilizar la libreria NLTK, asi como otras herramientas.

- De **NLTK** usaremos el *Lemmatizer* que aplica lematización a las palabras. Simplificando mucho, un lematizador convierte las palabras a su forma básica, por ejemplo las palabras "perro", "perros" denotan lo mismo, asi que solo deja la raiz de la palabra. Esto tiene el efecto de reducir el tamaño del diccionario.

- Tambien usaremos **BeautifulSoup** porque los datos vienen en XML y necesitamos extraerlos.

- Por ultimo, para la clasificación usaremos el típico clasificador de **Regresión Logistica** de sklearn. La clasifiación de sentimientos es binaria, positivo o negativo.

In [17]:
import nltk
import numpy as np

from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression
from bs4 import BeautifulSoup

In [18]:
# Instanciamos un objeto de la claseWordNetLemmatizer()
wordnet_lemmatizer = WordNetLemmatizer()


In [19]:
# Cargamos las 'stopwords' del idioma Ingles de algun sitio de internet
# from http://www.lextek.com/manuals/onix/stopwords1.html
stopwords = set(w.rstrip() for w in open('stopwords.txt'))

### Tenemos 3 archivos con las reseñas o criticas. 2 de ellos están etiquetados como POSITIVAS o NEGATIVAS. 

### El 3ro está sin etiquetar.

In [20]:
# Cargamos las criticas POSITIVAS
# data courtesy of http://www.cs.jhu.edu/~mdredze/datasets/sentiment/index2.html
positive_reviews = BeautifulSoup(open('/home/jorge/data/sorted_data_acl/electronics/positive.review').read())

In [21]:
positive_reviews.review_text

<review_text>\nI purchased this unit due to frequent blackouts in my area and 2 power supplies going bad.  It will run my cable modem, router, PC, and LCD monitor for 5 minutes.  This is more than enough time to save work and shut down.   Equally important, I know that my electronics are receiving clean power.\n\nI feel that this investment is minor compared to the loss of valuable data or the failure of equipment due to a power spike or an irregular power supply.\n\nAs always, Amazon had it to me in &lt;2 business days\n</review_text>

In [22]:
# Dejamos solo las del campo 'review_text'
positive_reviews = positive_reviews.findAll('review_text')

In [23]:
# por ejemplo la 10ma reseña positiva sería:
positive_reviews[10]

<review_text>\nI am very happy with this product. It folds super slim, so traveling with it is a breeze! Pretty good sound - not Bose quality, but for the price, very respectable! I've had it almost a year, and it has been along on many weekend get-aways, and works great. I use it alot, so it was a good purchase for me\n</review_text>

In [24]:
# Hacemos lo mismo con las Negativas
negative_reviews = BeautifulSoup(open('/home/jorge/data/sorted_data_acl/electronics/negative.review').read())
negative_reviews = negative_reviews.findAll('review_text')

In [25]:
negative_reviews[3]

<review_text>\nI knew these were inexpensive CD cases, but I can't even open one without it breaking into two pieces..\n</review_text>

In [26]:
# Tenemos muchas más revisiones POSITIVAS que NEGATIVAS, asi que tenemos un problema de balance

# Tomaremos una muestra de las positivas del mismo tamaño que las negativas, 
# asi tenemos las clases balanceadas
positive_reviews = positive_reviews[:len(negative_reviews)]


## Creación del Diccionario

El diccionario debe incluir todas las palabras de nuestro vocabulario. 
Para ello tenemos que centrarnos en las 'palabras' que son representativas y utiles. 

Por lo tanto necesitamos:

- tokenizar (eliminando distinciones entre mayusculas y minusculas) y luego 
- lemmatizar para dejar las palabras en su forma basica.



### Primero vamos a tokenizar los textos usando el tokenizador de NLTK.

In [33]:
# A ver como tokeniza la 2da critica
t = positive_reviews[2]
nltk.tokenize.word_tokenize(t.text)

[u'Wish',
 u'the',
 u'unit',
 u'had',
 u'a',
 u'separate',
 u'online/offline',
 u'light',
 u'.',
 u'When',
 u'power',
 u'to',
 u'the',
 u'unit',
 u'is',
 u'missing',
 u',',
 u'the',
 u'single',
 u'red',
 u'light',
 u'turns',
 u'off',
 u'only',
 u'when',
 u'the',
 u'warning',
 u'sounds',
 u'.',
 u'The',
 u'warning',
 u'sound',
 u'is',
 u'like',
 u'a',
 u'lot',
 u'of',
 u'sounds',
 u'you',
 u'hear',
 u'in',
 u'the',
 u'house',
 u'so',
 u'it',
 u'is',
 u"n't",
 u'always',
 u'easy',
 u'to',
 u'tell',
 u'what',
 u'is',
 u'happening']

Vemos que la tokenización está considerando como distintas las palabras minusculas y mayusculas. Entonces debemos dejar todo en minusculas para evitar que el diccionario crezca mucho.

Vamos a crear una función para tokenizar mejor.

La funcion my_tokenizer( ) hace:
- Pasa la reseña a minusculas
- Tokeniza la reseña (es decir separa el parrafo en palabras)
- Elimina las palabras de menos de 3 letras (in, a, by, ...)
- Elimina las pabras que están en la lista de 'stopwords'
- Devuelve una lista filtrada de palabras de la reseña

In [27]:
def my_tokenizer(s):
    s = s.lower() # todo en minusculas
    tokens = nltk.tokenize.word_tokenize(s) # separamos el string en "palabras" (tokens)
    tokens = [t for t in tokens if len(t) > 2] # eliminamos las palabras cortas, probablemente no son utiles
    tokens = [wordnet_lemmatizer.lemmatize(t) for t in tokens] # Lematizamos --> palabras en forma base
    tokens = [t for t in tokens if t not in stopwords] # quitamos stopwords
    return tokens

### Ahora si podemos crear el diccionario...

In [39]:
# Recordamos como funciona un diccionario en Python
# El diccionario es un conjuto de pares K:V donde K es la palabra y V el indice
diccionario ={}
a='uno'
b='dos'
c='tres'

# por ejemplo puedo hacer
diccionario[a]=1
diccionario[b]=2
diccionario[c]=3

print diccionario

{'dos': 2, 'tres': 3, 'uno': 1}


![diccionario2](diccionario2.jpg)

Creamos un mapa word-to-index de tal manera que podamos crear nuestros vectores de frecuencias mas tarde.
Tambien guardamos la tokenizacion en un par de listas para no tenerla que hacer mas adelante.

In [82]:
# Reseñas POSITIVAS
word_index_map = {}  # Creamos un diccionario vacio
current_index = 0
positive_tokenized = []
negative_tokenized = [] # Creamos un par de listas vacias

i=0 # cuento tokens



### OJO: positive_tokenized es una lista de listas. Es decir cada reseña tokenizada
###     crea una sublista dentro de la lista de reseñas positivas. Lo mismo luego con las negativas.
  

for review in positive_reviews:         # recorremos la lista de las reseñas POSITIVAS
    tokens = my_tokenizer(review.text)       # tokenizamos el texto de la reseña (devuelve lista)
    positive_tokenized.append(tokens)        # metemos la lista de palabras de la reseña tokenizada 
                                             # a la lista correspondiente
    i=i+1
    
    
    for token in tokens:                     # por cada token de la reseña
        if token not in word_index_map:            # verifico si NO esta en el diccionario
            word_index_map[token] = current_index       # entonces lo incluyo la palabra en el dicc y su indice
            current_index += 1                          # e incremento el contador
            
print "numero de reseñas positivas procesadas= ",i
print "Se han cargado hasta ahora ", current_index, " palabras/tokens en el diccionario."

numero de reseñas positivas procesadas=  1000
Se han cargado hasta ahora  7566  palabras/tokens en el diccionario.


In [84]:
# Hago lo mismo pero para las NEGATIVAS, pero en el mismo diccionario
for review in negative_reviews:
    tokens = my_tokenizer(review.text)
    negative_tokenized.append(tokens)
    for token in tokens:
        if token not in word_index_map:
            word_index_map[token] = current_index
            current_index += 1

In [86]:
print "Nuestro diccionario ahora tiene:", len(word_index_map), "palabras/tokens de las reseñas positivas y negativas."

Nuestro diccionario ahora tiene: 11088 palabras/tokens de las reseñas positivas y negativas.


In [71]:
todo_diccionario= word_index_map.items
todo_diccionario()

[(u'raining', 7126),
 (u'1600mah', 9819),
 (u'conspiratively', 8575),
 (u'yellow', 1630),
 (u'gag', 9107),
 (u'circuitry', 5506),
 (u'hanging', 1861),
 (u'centimeter', 9879),
 (u'marching', 4296),
 (u'shure', 2493),
 (u'looking', 1202),
 (u'accupower', 5014),
 (u'eligible', 7472),
 (u'electricity', 1447),
 (u'scold', 6548),
 (u'unanswered', 8517),
 (u'menus/programming', 8199),
 (u'superficially', 4079),
 (u'xtc', 4936),
 (u'crossbar', 8189),
 (u'sputter', 3783),
 (u'lord', 892),
 (u'swivel', 2355),
 (u'shielding', 8448),
 (u'why..', 6031),
 (u'dell', 1229),
 (u'hdtv', 902),
 (u'replaces', 8257),
 (u'foul', 10424),
 (u'malfunctioned', 2203),
 (u'uleads', 7644),
 (u'bringing', 4905),
 (u'+/-', 4394),
 (u'reformatted', 8103),
 (u'customizable', 1534),
 (u'account/billing', 11064),
 (u'microphone/recording', 6098),
 (u'persisted', 9626),
 (u'stethascope', 9621),
 (u'succession', 261),
 (u'reliable', 144),
 (u'eagle', 542),
 (u'preface', 6137),
 (u'existance', 5419),
 (u'270', 9803),
 (u'e

In [87]:
len(todo_diccionario())

11088

![diccionario3](diccionario3.jpg)

### Si el diccionario tiene esta longitud, significa que nuestra representación vectorial al menos tiene este número de dimensiones.

## Creación de las matrices de Entrada

### Cual es el tamaño de nuestros datos de entrada??

Una cosa es el tamaño del diccionario que define el numero de dimensiones de la representación vectorial y otra es el tamaño de las muestras, en este caso definido por el numero total de reseñas (positivas + negativas).

Como hicimos un balance de reseñas, tenemos 2.000 en total.

In [68]:
# Numero de reseñas tokenizadas de las listas de reseñas positivas y negativas, entonces:

N = len(positive_tokenized) + len(negative_tokenized)

N

2000

### Creamos la matriz de los datos de entrada

In [88]:
# podemos ver que es una lista de listas
for tokens in positive_tokenized:
    print tokens

[u'purchased', u'this', u'unit', u'due', u'frequent', u'blackout', u'power', u'supply', u'bad', u'run', u'cable', u'modem', u'router', u'lcd', u'monitor', u'minute', u'this', u'time', u'save', u'shut', u'equally', u'electronics', u'receiving', u'clean', u'power', u'feel', u'this', u'investment', u'minor', u'compared', u'loss', u'valuable', u'data', u'failure', u'equipment', u'due', u'power', u'spike', u'irregular', u'power', u'supply', u'amazon', u'business', u'day']
[u'apc', u'back-ups', u'500', u'recommendation', u'employee', u'mine', u'apc', u"'ve", u'month', u"'ve", u'functioned', u'properly', u'unexpected', u'power', u'interruption', u"'ll", u'gladly', u'arises', u'pro', u'plug', u'spacing', u'power', u'adapter', u'simple', u'design', u'cord', u'con', u'line', u'conditioning', u'usually', u'expensive', u'option']
[u'wish', u'unit', u'separate', u'online/offline', u'light', u'power', u'unit', u'missing', u'single', u'red', u'light', u'warning', u'sound', u'warning', u'sound', u'lot

In [89]:
for tokens in negative_tokenized:
    print tokens

[u'con', u'tip', u'extremely', u'easy', u'carpet', u'you', u'lot', u'cd', u'stacked', u'top', u'poorly', u'designed', u'vertical', u'rack', u'doesnt', u'individual', u'slot', u'cd', u'you', u'bottom', u'stack', u'you', u'basically', u'pull', u'stack', u'putting', u'wa', u'pain', u'bought', u'break', u'piece', u'metal', u'fit', u'guide', u'hole', u'again..poorly', u'designed', u'...', u'doesnt', u'fit', u'cd', u'gap', u'ca', u'loose', u'fitting', u'pro', u'...', u'...', u'...', u'guess', u'hold', u'lot', u'cd', u'...']
[u'nice', u'look', u'tip', u'easily', u'steady', u'rug', u'surface', u'dispite', u'picture', u'box', u'advice', u'you', u'rack', u'hold', u'lot', u'save', u'money', u'invest', u'nicer', u'sturdy']
[u'bought', u'returned', u'unit', u'ha', u'defective', u'finally', u'returning', u'system', u'dvd', u'player', u'constantly', u'bad', u'disc', u'error', u'skip', u'slightest', u'smudge', u'disc', u'sound', u'quality', u'nice', u'price', u'player', u'doe', u"n't", u'essentially',

In [90]:
# Creamos una función que transforma tokens en vectores

# Recibe una lista de palabras o tokens de la reseña y a etiqueta (en este caso positiva o negativa)
# Para cada palabra
#       busca el indice que le corresponde en el diccionario
#       incrementa el elemento del vector que va a representar a la reseña
#
# Una vez a recorrido todas las palabras de la reseña, normaliza los valores del vector.

# El vector que queda es un vector bastante sparse con el peso de las palabras de la reseña 
# y la etiqueta correspondiente

def tokens_to_vector(tokens, label):

    x = np.zeros(len(word_index_map) + 1) # creamos un vector de nulos del tamaño del diccionario
                                          # y agregamos 1 para la etiqueta
    
    for t in tokens:                      # recorremos los tokens de la reseña
        i = word_index_map[t]                  # guardo el indice de la palabra actual en i
        x[i] += 1                              # incremento el contador del vector x[] en 1

    # Al final del proceso, me queda un vector con el numero de repeticiones de cada palabra en la posición que 
    # tiene en el diccionario.

    # Ahora normalizamos de tal manera que todo sume 1 
    x = x / x.sum()                       # normalizamos antes de asignar la etiqueta
    x[-1] = label
    return x

In [92]:
# La matriz de entrada será entonces de N(numero de reseñas) x D(tamaño del diccionario)+1 (para la etiqueta) 

# (N x D+1) - las mantenemos juntas para poder hacer shuffle despues

# primero creamos la matriz con ceros
data = np.zeros((N, len(word_index_map) + 1))  ## N x D+1 (mio)
resenia = 0

# para cada elemento en la lista de positive_tokenized() la transformo en un vector
for tokens in positive_tokenized:
    xy = tokens_to_vector(tokens, 1)
    data[resenia,:] = xy
    resenia += 1
# para cada elemento en la lista de negative_tokenized() la transformo en un vector
for tokens in negative_tokenized:
    xy = tokens_to_vector(tokens, 0)
    data[resenia,:] = xy
    resenia += 1

In [93]:
data.shape

(2000, 11089)

Recordamos: 2000 reseñas x 11.089 palabras eb el diccionario

Los vectores que representan reseñas positivas deben sumar 2 (los datos normalizados  suman 1 + 1 de la etiqueta)

In [98]:
data[1,:].sum()

2.0

Mientras que los vectores que representan reseñas negativas deben sumar 1.

In [99]:
data[1999,:].sum()

1.0

## Entrenamos con los datos que hemos preparado

Nos ha quedado una matriz de 2000x11086+1 cuyas filas representan a las reseñas y su etiqueta.

In [127]:
# Hacemos shuffle de los datos, separamos los datos de entrada X() de la etiqueta en Y() 
# y creamos datos de train/test 

np.random.shuffle(data)

X = data[:,:-1]
Y = data[:,-1]

X.shape

(2000, 11088)

In [128]:
Y.shape

(2000,)

In [129]:
# Por ejemplo, tomamos las ultimas 100 para test
muestras_train=-100
Xtrain = X[:muestras_train,]
Ytrain = Y[:muestras_train,]
Xtest = X[muestras_train:,]
Ytest = Y[muestras_train:,]

In [130]:
# Vamos a aplicar una Regresion Logistica tradicional
model = LogisticRegression()
model.fit(Xtrain, Ytrain)
print "Scoring de clasificacion:", model.score(Xtest, Ytest)


Scoring de clasificacion: 0.73


model.score(Xtest, Ytest) - Returns the mean accuracy on the given test data and labels.
In multi-label classification, this is the subset accuracy which is a harsh metric since you require for each sample that each label set be correctly predicted.

In [131]:
model.coef_.shape

(1, 11088)

In [132]:
print "coef. max.:",model.coef_.max(), " ------------      coef. min.:",model.coef_.min(), 

coef. max.: 2.53078478324  ------------      coef. min.: -1.95248412992


### Ahora podemos ver el peso de las palabras individualmente

La idea es ver qué palabras tienen pesos claramente positivos o negativos.

Una vez se ha entrenado el modelo, el vector de coeficientes del modelo (en este caso es de 1x11.088) guarda los pesos de cada una de las palabras. Las que son >0 se consideran positivas.  

Para esto ponemos un umbral y vemos cuales quedan por encima del umbral y por debajo del umbral negativo.

In [137]:
#Veamos los pesos de cada palabra
threshold = 1
for word, index in word_index_map.iteritems():
    weight = model.coef_[0][index]
    if weight > threshold or weight < -threshold:
        print "[",index,"]", word, weight

[ 79 ] easy 1.91264689125
[ 277 ] love 1.11296929845
[ 124 ] wa -1.66174910342
[ 139 ] price 2.53078478324
[ 75 ] you 1.13316425119
[ 96 ] quality 1.45269066653
[ 198 ] doe -1.17709579878
[ 78 ] n't -1.95248412992
[ 545 ] then -1.13452718942
[ 143 ] money -1.19354004021
[ 273 ] excellent 1.34190468899
[ 2803 ] return -1.21999623995
